In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
from csv import DictReader #untuk membuat csv jadi sebuah data set untuk dianalisa
from typing import List,Dict #nantinya buat menyimpan data dict di dalam list untuk di sum nantinya
import pandas as pan
import pymongo
from pymongo import MongoClient


# Scraping Function

In [ ]:
def Scrap(): 
    url = " " #input URL e-commerce
    headers = {
        'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'
    }
    datas = []
    datas2 = [] #buat harga yang sudah di convert ke int
    datas3 = [] #buat database nama barang dan harga
    count_page = 0
    for page in range(1,4):
        count_page += 1
        print("halaman terscraping : ",count_page)
        req = requests.get(url+str(page),headers=headers)
        soup = BeautifulSoup(req.text,'html.parser')
        items = soup.findAll('  ',' ') #find in BS documentation
        for i in items :
            nama = i.find('  ','  ').text #find in BS documentation
            harga = i.find('  ','  ').text #find in BS documentation
            datas.append([nama, harga])
            print(harga) #ini masih string
            harga2 = harga.replace('Rp','') #proses untuk mengubah string ke int di python harus manual dengan menghilangkan unsur2 stringnya
            harga3 = harga2.replace('.','') #berhubung string di sini sama 'Rp xx.xxx.xxx' jadi hilangkan Rp, dan . saja
            intHarga = int(0)
            intHarga = harga3.replace(' ','')
            print(intHarga) 
            datas2.append([nama,intHarga])
            datas3.append([intHarga]) #ini nanti untuk perbandingan csv file yg Rp. nya sudah di convert ke INT
    header = ['Nama', 'Harga']
    writer = csv.writer(open('YourFileLocation.csv'.format(" Title "),'w',newline='')) 
    writer.writerow(header)
    for i in datas:
        writer.writerow(i)
    # di bawah ini untuk nanti di mongoDB
    header2 = ['Nama','Harga (IDR)']
    writer = csv.writer(open('YourFileLocation.csv'.format("Title"),'w',newline='')) 
    writer.writerow(header2)
    for i in datas2:
        writer.writerow(i)
    header3 = ['Harga (IDR)'] #ini buat jadi data Harga yg sudah jadi integer isinya harga saja
    writer = csv.writer(open('YourFileLocation.csv'.format("Title"),'w',newline='')) #untuk yg currency dalam int untuk di analisa
    writer.writerow(header3)
    for i in datas3:
        writer.writerow(i)

# Average function

In [ ]:
def rata(): #function to create file
    with open("Title","r") as file:
        fharga = pan.read_csv(file)
        rata = fharga["Harga (IDR)"].mean()
        return rata
    file.close()

def RataHarga(): #function for average
    bacafile = open('YourFileLocation.csv','r')
    baca_csv = DictReader(bacafile) #tabelharga: adalah genericAlias jadi bukan tabelharga = tapi tabelharga :
    tabelharga : List[Dict[int, int]] = [] #ini penggunaan yang module typing diatas, isi list hanya ada harga tapi tapi parameter minimal 2 jadi saya masukan saja int lagi meski tidak ada
    for row in baca_csv : #row ini sebuah dict dari file csv diatas
        dictData: Dict[int, int] = {} #ini untuk tiap kolom di dalam baris csv Harga
        for column in row:
            dictData[column] = int(row[column])
        tabelharga.append(dictData)
    print(tabelharga) #mencoba apakah berhasil membuat list of dict "tabelharga", agar bisa di sum dan dicari averagenya
    harga_sum = int(0)
    for row in tabelharga : 
        harga_sum += row['Harga (IDR)']
    rataharga = harga_sum/len(tabelharga)
    return rataharga
    bacafile.close()

# MongoDB Implementation

In [ ]:
koleksirata = input("Masukkan nama untuk collection rata-rata : ")
def insertRata():
    cluster = MongoClient("mongodb://localhost:xxxxx/")
    db = cluster["Cluster name"]
    collection = db[koleksirata] 
    bacafile = open('YourFileLocation.csv','r')
    baca_csv = DictReader(bacafile)
    tabelharga: List[Dict[int, int]] = [] 
    for row in baca_csv : 
        dictData: Dict[int, int] = {} 
        for column in row:
            dictData[column] = int(row[column])
        tabelharga.append(dictData)
    collection.insert_many(tabelharga) 
    ratarata=collection.aggregate([ #Dari mongoDB docs
        {"$group": {"_id": " " ,"Rata-rata Harga (MongoDB)": {"$avg": "$Harga (IDR)"}}}
    ])
    for i in ratarata: 
        print(i)
    bacafile.close()

koleksisort_search = input("Masukkan nama untuk collection sorting dan searching : ")  
def insertSorting():
    cluster = MongoClient("mongodb://localhost:xxxxx/")
    db = cluster["Cluster name"]
    collection = db[koleksisort_search]
    with open('YourFileLocation.csv','r') as databarang:
        csv_read = DictReader(databarang)
        tabel = [] #datanya masih string semua
        for row in csv_read:
            tabel.append(row)
    collection.insert_many(tabel)
    databarang.close()
    print("Urutkan :\n1.Termurah\n2.Termahal")
    urut = int(input("Masukan (1/2) : "))
    if urut == 1 :  
        print("Berikut hasil sorting dari termurah :")
        urutan = collection.find({},{"_id":0}).sort("Harga (IDR)",pymongo.DESCENDING) 
        for x in urutan:
            print (x)
    elif urut == 2 :
        print("Berikut hasil sorting dari Termahal :")
        urutan = collection.find({},{"_id":0}).sort("Harga (IDR)",pymongo.ASCENDING)
        for x in urutan:
            print (x)
    else :
        print("Masukkan salah")
    databarang.close()

def cariMongoDB():
    cluster = MongoClient("mongodb://localhost:xxxxx/")
    db = cluster["cluster name"]
    collection = db[koleksisort_search]
    masukancari = input("Masukan pencarian : ")
    temukan = {"Nama":{"$regex":masukancari}}
    temu = collection.find(temukan)
    print("Berikut hasil pencarian anda : ")
    for i in temu:
        print(i)

In [ ]:
Scrap()
rata_rataPanda = rata()
rata_rataTyping = RataHarga()
print("\n")
print("\nRata-rata harga barang(Module Panda)  : Rp.",rata_rataPanda)
print("Rata-rata harga barang(Module Typing) : Rp.",rata_rataTyping)
insertRata()
print("\n")
insertSorting()
print("\n")
cariMongoDB()